<a href="https://colab.research.google.com/github/JacopoBartoli/vas_regression/blob/main/dataset_generation_BioVid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Organize imports

In this section we install and import the needed packages. Then we mount our GDrive.

In [ ]:
import pandas as pd
import numpy as np

Useful paths.

In [ ]:
# Name of file with data for create dataset
file_name = 'dataset_final_1700.csv'

# Path of two part of dataset
COORD_DF_PATH = 'path_to_file' + file_name  # Path to file dataset_final
SEQ_DF_PATH = '/path_to_file_labels/labels_BioVid.csv' # Path to file labels_BioVid 

# Path where save the data extract from dataset
DATASET_DIR = '/path_to_save/'



Mount the drive.

In [ ]:
# Mount your drive to access the dataset.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 2) BioVid Heat Pain Dataset Generation

In this section we extract the data from dataset.

## 2.1) Set parameters


In [ ]:
# List of index of landmarks to be taken into consideration

selected_lndks_idx = range(0,70)

num_lndks = len(selected_lndks_idx)

# Index of central landmark of the frame

central_lndk = 30


##2.2) Utility functions

Define some utilities functions.

In [ ]:
# Get the velocities of all selected landmark for each frame of each sequence

def get_velocities_frames():
    coord_df = pd.read_csv(COORD_DF_PATH)
    seq_df = pd.read_csv(SEQ_DF_PATH)
    velocities = []

    list_seq = coord_df['0'].unique()

    for seq_num in list_seq:

        vel_seq = []

        lndks = coord_df.loc[coord_df['0'] == seq_num].values

        lndks = lndks[:, 2:]

        # Start frame

        frame = lndks[0]

        frame_nose_x = frame[central_lndk]
        frame_nose_y = frame[central_lndk + num_lndks]


        frame_nose = []

        for i in range(num_lndks):
            frame_nose.append(frame_nose_x)
        for i in range(num_lndks):
            frame_nose.append(frame_nose_y)

        frame_centered_prec = frame - frame_nose

        for id_frame in range(1,len(lndks)):

            frame = lndks[id_frame]

            frame_centered_succ = frame - frame_nose

            frame_centered_succ_array = np.array(frame_centered_succ)
            frame_centered_prec_array = np.array(frame_centered_prec)

            # Velocity of landmarks

            lndk_vel = np.power(np.power(frame_centered_succ_array[0:num_lndks]-frame_centered_prec_array[0:num_lndks], 2) +
                               np.power(frame_centered_succ_array[num_lndks:]-frame_centered_prec_array[num_lndks:], 2), 0.5)

            vel_seq.append(np.array(lndk_vel))


            frame_nose_x = frame[central_lndk]
            frame_nose_y = frame[central_lndk + num_lndks]

            frame_nose = []

            for i in range(num_lndks):
                frame_nose.append(frame_nose_x)
            for i in range(num_lndks):
                frame_nose.append(frame_nose_y)

            frame_centered_prec = frame - frame_nose

        velocities.append(vel_seq)


    return velocities


## 2.3) Select train or test dataset generation

In [ ]:
# Train

name_csv = DATASET_DIR + 'all-train-velocity-' + str(len(selected_lndks_idx)) + '-' + file_name


In [ ]:
# Test

name_csv = DATASET_DIR + 'all-test-velocity-' + str(len(selected_lndks_idx)) + '-' + file_name


In [ ]:
# Create a csv file, for the training or test dataset

velocities = get_velocities_frames()
seq_df = pd.read_csv(SEQ_DF_PATH)


frame = []
lst = []
element = []
sequenza = []
for id_seq in range(0, len(velocities)):
    vas = seq_df.iloc[id_seq][1]
    element.append(id_seq)
    sequenza = velocities[id_seq]
    for id_frames in range(0, len(sequenza)):
        element.append(id_frames)
        frame = sequenza[id_frames]
        for v in range(0, len(frame)):
            velocita = frame[v]
            element.append(velocita)
        element.append(vas)
        lst.append(element)
        element = [id_seq]
    element = []



col = ['Sequenza','Frame']
for i in range(0, len(selected_lndks_idx)):
    s = 'Vel' + str(i)
    col.append(s)
col.append('Label')

df = pd.DataFrame(lst,columns=col)
print(df)

df.to_csv('path_to_save' + name_csv, index=False)


        Sequenza  Frame      Vel0  ...     Vel68     Vel69  Label
0              0      0  1.665200  ...  0.262088  0.326535      0
1              0      1  2.955522  ...  1.154735  0.437444      0
2              0      2  2.261649  ...  0.308423  1.376483      0
3              0      3  2.468882  ...  2.307309  0.712849      0
4              0      4  0.447599  ...  1.512842  1.290358      0
...          ...    ...       ...  ...       ...       ...    ...
136995       999    132  3.246357  ...  1.234049  0.899861      4
136996       999    133  3.874190  ...  1.454432  1.945173      4
136997       999    134  5.654878  ...  0.848559  1.604639      4
136998       999    135  5.105019  ...  2.018626  5.264548      4
136999       999    136  1.284234  ...  1.079114  3.186679      4

[137000 rows x 73 columns]
Salvato
